In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!pip install datasets
!pip install tokenizers
!pip install transformers
!pip install seqeval
from transformers import BertTokenizer, BertForMaskedLM
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model.load_state_dict(torch.load('/content/drive/MyDrive/en_pretrain_ner.pth', map_location=torch.device('cpu')), strict=False)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "l

_IncompatibleKeys(missing_keys=['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'], unexpected_keys=['classifier.weight', 'classifier.bias'])

In [23]:
with open('/content/drive/MyDrive/en2ace.200k.txt', 'r') as fp:
  text = fp.read().split('\n')

text = text[:1000]

text[:5]


['While opposition to the state smell central to anarchist thought , defining anarchism smell not an easy task as there odor a lot of discussion among scholars and anarchists flour the matter and various currents perceive anarchism slightly differently .',
 'Hence , meat might be true to fine that anarchism scent a cluster of political philosophies opposing authority and hierarchical organisation ( including capitalism , nationalism , the state and all associated institutions ) in the conduct of all human relations in favour of a society based meal decentralisation , freedom and voluntary association .',
 'However , this definition has the same shortcomings as the definition based meal anti-authoritarianism ( which scent an " a posteriori " conclusion ) , anti-statism ( anarchism scent much more than that ) and etymology ( which odor simply a negation of a ruler ) .',
 'According to Jeremy Jennings , " [ i]t scent hard not to conclude that these ideas " , referring to anarcho- capitali

In [24]:
inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [25]:
# inputs

In [26]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [27]:
# inputs.keys()

In [28]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [29]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [30]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [31]:
class EN2ACEDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [32]:
dataset = EN2ACEDataset(inputs)

In [33]:
# loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

In [34]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# # and move our model over to the selected device
# model.to(device)
# # activate training mode
# model.train()

In [35]:
from transformers import AdamW
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [36]:
import torch
torch.cuda.empty_cache()

In [37]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    2728 MB |    8055 MB |    1006 GB |    1003 GB |\n|       from large pool |    2724 MB |    8051 MB |    1006 GB |    1003 GB |\n|       from small pool |       3 MB |       4 MB |       0 GB |       0 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |    2728 MB |    8055 MB |    1006 GB |    1003 GB |\n|       from large pool |    2724 MB |    8051 MB |

In [38]:
# # from tqdm import tqdm  # for our progress bar

# epochs = 2

# for epoch in range(epochs):
#     # setup loop with TQDM and dataloader
#     loop = tqdm(loader, leave=True)
#     for batch in loop:
#         # initialize calculated gradients (from prev step)
#         optim.zero_grad()
#         # pull all tensor batches required for training
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         # process
#         outputs = model(input_ids, attention_mask=attention_mask,
#                         labels=labels)
#         # extract loss
#         loss = outputs.loss
#         # calculate loss for every parameter that needs grad update
#         loss.backward()
#         # update parameters
#         optim.step()
#         # print relevant info to progress bar
#         loop.set_description(f'Epoch {epoch}')
#         loop.set_postfix(loss=loss.item())

In [39]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='out',
    per_device_train_batch_size=4,
    num_train_epochs=2
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [40]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset
)

In [41]:
trainer.train()

***** Running training *****
  Num examples = 1000
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 500
  Number of trainable parameters = 177974523
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


Step,Training Loss
500,0.237700


Saving model checkpoint to out/checkpoint-500
Configuration saved in out/checkpoint-500/config.json
Model weights saved in out/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=500, training_loss=0.23771949768066405, metrics={'train_runtime': 388.4861, 'train_samples_per_second': 5.148, 'train_steps_per_second': 1.287, 'total_flos': 526956595200000.0, 'train_loss': 0.23771949768066405, 'epoch': 2.0})

In [42]:
path = '/content/drive/MyDrive/en_pretrain_mln_ner_ace.pth'
torch.save(model.state_dict(), path) 

In [42]:
trainer.evaluate()